In [ ]:
import cv2
import time
import numpy as np
from ultralytics import YOLO
import math as m

# Initialize YOLOv8 pose estimation model
model = YOLO('yolov8n-pose.pt')

# Posture thresholds
POSTURE_THRESHOLDS = {
    'back_angle': 20,        # Degrees from vertical
    'face_distance': 0.15,   # Ratio of image height
    'hip_level': 0.1,        # Ratio of image height
    'neck_inclination': 40,  # Degrees
    'torso_inclination': 10  # Degrees
}

# Keypoint indices (COCO format)
KEYPOINTS = {
    'NOSE': 0,
    'LEFT_SHOULDER': 5,
    'RIGHT_SHOULDER': 6,
    'LEFT_HIP': 11,
    'RIGHT_HIP': 12
}

# Visualization settings
COLORS = {
    'good': (0, 255, 0),
    'bad': (0, 0, 255),
    'warning': (0, 255, 255),
    'metrics': (127, 255, 0)
}

def calculate_vertical_angle(pt1, pt2):
    """Calculate angle from vertical (degrees)"""
    dx = pt2[0] - pt1[0]
    dy = pt2[1] - pt1[1]
    return np.degrees(np.arctan2(abs(dx), abs(dy)))

def find_angle(x1, y1, x2, y2, x3, y3):
    """Calculate angle between three points"""
    angle = m.degrees(m.atan2(y3 - y2, x3 - x2) - m.atan2(y1 - y2, x1 - x2))
    return angle + 360 if angle < 0 else angle

def posture_analysis(keypoints, frame_height):
    """Analyze posture for a single person"""
    results = {
        'back_angle': 0,
        'face_distance': 0,
        'hip_level': 0,
        'neck_inclination': 0,
        'torso_inclination': 0,
        'messages': []
    }

    # Get required keypoints
    kpt = KEYPOINTS
    ls = keypoints[kpt['LEFT_SHOULDER']]
    rs = keypoints[kpt['RIGHT_SHOULDER']]
    lh = keypoints[kpt['LEFT_HIP']]
    rh = keypoints[kpt['RIGHT_HIP']]
    nose = keypoints[kpt['NOSE']]

    # Calculate midpoints
    shoulder_mid = ((ls[0] + rs[0])/2, (ls[1] + rs[1])/2)
    hip_mid = ((lh[0] + rh[0])/2, (lh[1] + rh[1])/2)

    # Back angle (shoulders to hips)
    results['back_angle'] = calculate_vertical_angle(shoulder_mid, hip_mid)
    if results['back_angle'] > POSTURE_THRESHOLDS['back_angle']:
        results['messages'].append("Straighten your back")

    # Face distance from shoulders
    results['face_distance'] = abs(nose[1] - shoulder_mid[1]) / frame_height
    if results['face_distance'] > POSTURE_THRESHOLDS['face_distance']:
        results['messages'].append("Align head with shoulders")

    # Hip level alignment
    results['hip_level'] = abs(lh[1] - rh[1]) / frame_height
    if results['hip_level'] > POSTURE_THRESHOLDS['hip_level']:
        results['messages'].append("Balance hip position")

    # Neck inclination (nose-shoulder-horizontal)
    results['neck_inclination'] = find_angle(
        nose[0], nose[1], 
        shoulder_mid[0], shoulder_mid[1],
        shoulder_mid[0], shoulder_mid[1] - 100
    )
    if results['neck_inclination'] > POSTURE_THRESHOLDS['neck_inclination']:
        results['messages'].append("Keep head straight")

    # Torso inclination (shoulder-hip-horizontal)
    results['torso_inclination'] = find_angle(
        shoulder_mid[0], shoulder_mid[1],
        hip_mid[0], hip_mid[1],
        hip_mid[0], hip_mid[1] - 100
    )
    if results['torso_inclination'] > POSTURE_THRESHOLDS['torso_inclination']:
        results['messages'].append("Keep torso upright")

    return results

# Initialize video capture
cap = cv2.VideoCapture(0)
person_data = {}
last_update = time.time()

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    current_time = time.time()
    results = model(frame, verbose=False)[0]
    keypoints = results.keypoints.xy.cpu().numpy()
    h, w = frame.shape[:2]

    # Process all detected people
    for i, person_kpts in enumerate(keypoints):
        if i not in person_data:
            person_data[i] = {'duration': 0, 'last_seen': current_time}

        # Update posture duration
        time_diff = current_time - person_data[i]['last_seen']
        analysis = posture_analysis(person_kpts, h)
        
        if len(analysis['messages']) > 0:
            person_data[i]['duration'] += time_diff
        else:
            person_data[i]['duration'] = max(0, person_data[i]['duration'] - time_diff)
        
        person_data[i]['last_seen'] = current_time

        # Visualization
        y_offset = 30 + (i * 200)
        color = COLORS['bad'] if len(analysis['messages']) > 0 else COLORS['good']
        
        # Draw skeleton
        for kpt in person_kpts:
            x, y = int(kpt[0]), int(kpt[1])
            cv2.circle(frame, (x, y), 5, color, -1)

        # Display metrics
        cv2.putText(frame, f"Person {i+1}", (10, y_offset), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, 
                    f"Back: {analysis['back_angle']:.1f}° | Neck: {analysis['neck_inclination']:.1f}°", 
                    (10, y_offset+30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, COLORS['metrics'], 1)
        cv2.putText(frame, 
                    f"Hip Level: {analysis['hip_level']*100:.1f}% | Duration: {person_data[i]['duration']:.1f}s", 
                    (10, y_offset+60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, COLORS['metrics'], 1)

        # Display warnings
        for j, msg in enumerate(analysis['messages']):
            cv2.putText(frame, msg, (w-300, 30 + (j*30)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, COLORS['warning'], 1)

    # Cleanup old person data
    for pid in list(person_data.keys()):
        if current_time - person_data[pid]['last_seen'] > 2:
            del person_data[pid]

    cv2.imshow('Posture Monitoring', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()